In [12]:
#Part 4: Federated Learning with Model Averaging
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

In [24]:
#Step 1: Create Data Owners

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [25]:
#Step 2: Create Our Model

# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [26]:
#Step 3: Send a Copy of the Model to Alice and Bob
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [27]:
#Step 4: Train Bob's and Alice's Models (in parallel)
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0828) Alice:tensor(3.4268)
Bob:tensor(0.0344) Alice:tensor(0.1804)
Bob:tensor(0.0211) Alice:tensor(0.1296)
Bob:tensor(0.0162) Alice:tensor(0.1077)
Bob:tensor(0.0134) Alice:tensor(0.0896)
Bob:tensor(0.0113) Alice:tensor(0.0746)
Bob:tensor(0.0096) Alice:tensor(0.0621)
Bob:tensor(0.0082) Alice:tensor(0.0517)
Bob:tensor(0.0070) Alice:tensor(0.0430)
Bob:tensor(0.0060) Alice:tensor(0.0358)


In [28]:
#Step 5: Send Both Updated Models to a Secure Worker¶

alices_model.move(secure_worker)

bobs_model.move(secure_worker)
print(secure_worker._objects)

{8907096184: Parameter containing:
tensor([[0.4831, 0.1953]], requires_grad=True), 13063424338: Parameter containing:
tensor([0.4072], requires_grad=True), 10935801832: Parameter containing:
tensor([[0.1511, 0.0989]], requires_grad=True), 46175164514: Parameter containing:
tensor([-0.0609], requires_grad=True), 24846122218: Parameter containing:
tensor([[0.5275, 0.1584]], requires_grad=True), 40328432267: Parameter containing:
tensor([0.3835], requires_grad=True), 99118562399: Parameter containing:
tensor([[0.3171, 0.0224]], requires_grad=True), 48527867835: Parameter containing:
tensor([-0.0074], requires_grad=True), 56625690104: Parameter containing:
tensor([[0.5865, 0.1091]], requires_grad=True), 51608591515: Parameter containing:
tensor([0.3523], requires_grad=True), 13348844409: Parameter containing:
tensor([[ 0.4223, -0.0099]], requires_grad=True), 49700984658: Parameter containing:
tensor([0.0121], requires_grad=True), 93657438650: Parameter containing:
tensor([[0.6400, 0.0751]]

In [29]:
#Step 6: Average the Models
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [30]:
print(alices_model)
print(bobs_model)

Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)


In [9]:
#Rinse and Repeat
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0006) Alice:tensor(0.0153)
Bob:tensor(0.0004) Alice:tensor(0.0073)
Bob:tensor(0.0008) Alice:tensor(0.0034)
Bob:tensor(0.0012) Alice:tensor(0.0017)
Bob:tensor(0.0013) Alice:tensor(0.0008)
Bob:tensor(0.0013) Alice:tensor(0.0004)
Bob:tensor(0.0012) Alice:tensor(0.0002)
Bob:tensor(0.0010) Alice:tensor(0.0001)
Bob:tensor(0.0008) Alice:tensor(7.2784e-05)
Bob:tensor(0.0007) Alice:tensor(4.3843e-05)


In [10]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

print(preds)
print(target)
print(loss.data)

tensor([[0.0787],
        [0.0671],
        [0.9148],
        [0.9032]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0273)
